In [1]:
import pandas as pd
import os
import glob
def get_data(dn):
    data = {
        "news":[],
        "ans":[]
    }
    dirs = glob.glob(os.path.join(dn, "*"))
    for d in dirs:
        pat = os.path.join(d, "*.[tT][xX][tT]")
        fs = glob.glob(pat)
        for fp in fs:
            with open(fp, "r", encoding="utf-8") as f :
                news = f.read()
            ans = os.path.split(d)[-1]
            data["news"].append(news)
            data["ans"].append(ans)
    df = pd.DataFrame(data, columns=["news","ans"])
    return df

In [2]:
train_df = get_data("chinese_news_trans/chinese_news_trans")
test_df = get_data("chinese_news_test/chinese_news_test")
test_df

,news,ans
0,日月光華 -- Traffic_Info精華區文章閱讀\n-----------------...,交通
1,日月光華 -- Traffic_Info精華區文章閱讀\n-----------------...,交通
2,日月光華 -- Traffic_Info精華區文章閱讀\n-----------------...,交通
3,三趟火車停開 乘客可全額退票 \n瀏覽次數：1180 \n 昨日，來自鐵路部門的消息說...,交通
4,日月光華 -- Traffic_Info精華區文章閱讀\n-----------------...,交通
...,...,...
96,最優秀選手無緣亞運會健美賽\n \n \n 健美在亞洲運動會上是“新生兒”——韓國釜...,體育
97,\n各國記者眼中的羽毛球世錦賽\n \n\n-----------------------...,體育
98,\n友好運動會第五天 東道主選手大顯神威\n2001年09月03日02:51:40 新華社 ...,體育
99,不靠技術比運氣\n\n 第二屆奧運會在巴黎舉行，同時這裡也正在舉行國際博覽會，東道主把一些...,體育


In [22]:
# unique value_count
# .unique() 重複的刪除
kind = train_df["ans"].unique()
kind
# enumerate
trans = {k:i for i, k in enumerate(kind)}
reverse_trans = {i:k for i, k in enumerate(kind)}
k=[]
for i in kind:
    k=k+[i]
print(k)
trans

['交通', '政治', '教育', '環境', '經濟', '藝術', '計算機', '軍事', '醫藥', '體育']


{'交通': 0,
 '政治': 1,
 '教育': 2,
 '環境': 3,
 '經濟': 4,
 '藝術': 5,
 '計算機': 6,
 '軍事': 7,
 '醫藥': 8,
 '體育': 9}

In [24]:
y_train = train_df["ans"].replace(trans)
y_test = test_df["ans"].replace(trans)

In [26]:
import os
from urllib.request import urlretrieve
DICT_PATH = "dict.txt.big"
if not os.path.exists(DICT_PATH):
    print("字典不存在")
    url = "https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big"
    urlretrieve(url, DICT_PATH)

In [27]:
import jieba
jieba.set_dictionary(DICT_PATH)
def newscut(news):
    s = " ".join(jieba.cut(news))
    return s.replace("\r", "").replace("\n", "")
x_train_raw = train_df["news"].apply(newscut)
x_test_raw = test_df["news"].apply(newscut)

Building prefix dict from D:\code\PycharmProjects\machine_learning_0407\dict.txt.big ...
Loading model from cache C:\Users\sore\AppData\Local\Temp\jieba.u0eb74b94744750b7ed4acf4fc6132938.cache
Loading model cost 3.418 seconds.
Prefix dict has been built successfully.


In [29]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
x_train = vec.fit_transform(x_train_raw)
x_test = vec.transform(x_test_raw)

In [31]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(x_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [32]:
from sklearn.metrics import accuracy_score
pre = clf.predict(x_test)
accuracy_score(pre, y_test)

1.0

In [33]:
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(y_test, pre)
col = ["{}(預測)".format(k) for k in kind]
row = ["{}(答案)".format(k) for k in kind]
pd.DataFrame(mat, columns=col, index=row)

,交通(預測),政治(預測),教育(預測),環境(預測),經濟(預測),藝術(預測),計算機(預測),軍事(預測),醫藥(預測),體育(預測)
交通(答案),10,0,0,0,0,0,0,0,0,0
政治(答案),0,11,0,0,0,0,0,0,0,0
教育(答案),0,0,10,0,0,0,0,0,0,0
環境(答案),0,0,0,10,0,0,0,0,0,0
經濟(答案),0,0,0,0,10,0,0,0,0,0
藝術(答案),0,0,0,0,0,10,0,0,0,0
計算機(答案),0,0,0,0,0,0,10,0,0,0
軍事(答案),0,0,0,0,0,0,0,10,0,0
醫藥(答案),0,0,0,0,0,0,0,0,10,0
體育(答案),0,0,0,0,0,0,0,0,0,10


In [34]:
n = input("請輸入新聞:")
test = vec.transform([newscut(n)])
pre = clf.predict(test)[0]
print("應該是:", reverse_trans[pre])
proba = clf.predict_proba(test)[0]
proba = list(enumerate(proba))
for i, prob in sorted(proba, key=lambda x:x[1], reverse=True):
    print(reverse_trans[i], ":", prob * 100, "%")

請輸入新聞:新北市一名林姓男子上月騎機車在新店區中正路等候姊姊下班，準備一同返家，卻無預警遭王姓男子從後方朝背部猛刺一刀，送醫宣告不治。台北地檢署調查後，今依殺人罪起訴，考量王男犯後態度不佳，向法院請求量處最嚴厲之刑。  家住新北市中和區的32歲林姓男子，今年3月13日晚間10時許，騎機車至新店區中正路上一間連鎖超商前，才以手機傳完訊息告知姊姊，人已到樓下，在路旁等候即將下班的姊姊，突遭王姓男子（22歲）持料理刀從背後猛力捅了一刀，當場連人帶車倒地。  路人見狀報警送醫，王隨即轉身開車逃逸，林男送慈濟新店分院急救後宣告不治。目擊者王妻告知警方王開車往新店秀朗橋往中和方向，新店警方在案發後20分鐘，迅速趕到秀朗橋頭逮捕王帶回派出所。  警詢後將王依殺人罪移送北檢，檢方訊後以王涉殺人重罪、有逃亡之虞，向法院聲請羈押，法院裁准。檢方審酌王僅因與其妻因用餐地點意見不合，為發洩情緒、報復其妻不順從其意，隨機挑選不認識、無任何仇恨、過節之無辜被害人下手，堪認王心態可議，惡性重大。  檢方認為，王行凶後，林坐在人行道上看向王的背影，發出淒厲求救聲，臉上表情甚為驚恐，惟王竟視若無睹，立即駕車逃逸，可見其手段兇殘，林生前突遭陌生人背刺的精神恐懼及其身體痛楚，非文字所能形容，且令林男家屬承受驟失親人痛苦，造成無法彌補之損害。  王於本案發生後，始終否認具殺人犯意，藉詞脫罪，且尚未賠償林男家屬，犯罪後態度難認良好等節，暨王男智識、經濟能力等一切情狀，建請量處最嚴厲之刑。
應該是: 醫藥
醫藥 : 99.8151560095166 %
軍事 : 0.15672155918577627 %
體育 : 0.019555899255847088 %
交通 : 0.008546186581816606 %
環境 : 2.0344379739758302e-05 %
計算機 : 1.0198784549381766e-09 %
政治 : 7.091050716084977e-11 %
教育 : 2.972772307361938e-13 %
藝術 : 2.401476818612812e-13 %
經濟 : 2.4826298013655537e-22 %


In [23]:
list(zip([1,2,3],
         [7,8,9],
         [4,5,6]))

[(1, 7, 4), (2, 8, 5), (3, 9, 6)]

In [35]:
sorted([(3, "a"), (1, "b"), (5, "c")])

[(1, 'b'), (3, 'a'), (5, 'c')]